# 437. Path Sum III


## Topic Alignment
- MLE Connection: Cumulative feature attribution along decision trees requires quick prefix lookups when exploring candidate splits.
- Hash Table Role: Map running prefix sums to frequency counts so we can query prefixSum - target in O(1) during DFS.
- Interview Angle: Blends tree traversal with stateful hash maps, testing careful backtracking and duplicate counting avoidance.


## Metadata Summary
- Source: https://leetcode.com/problems/path-sum-iii/
- Tags: Tree, Depth-First Search, Hash Table
- Difficulty: Medium
- Recommended Review Priority: High


## Problem Statement
You are given the root of a binary tree and an integer targetSum. Return the number of paths where the sum of the values along the path equals targetSum. The path does not need to start at the root or end at a leaf, but it must always move downward from parent nodes to child nodes.


## Progressive Hints
- Hint 1: Track the running prefix sum on the current root-to-node path.
- Hint 2: If you know the current prefix sum, how can you count prior prefixes that form targetSum when paired with the current node?
- Hint 3: Maintain a hash map of prefixSum -> count and update it as you descend and backtrack in DFS.


## Solution Overview
Traverse the tree with DFS while keeping a running prefix sum. For every node we need the number of earlier prefixes whose value equals currentPrefix - targetSum. A hash map keyed by prefix sum counts gives us that answer in O(1). We increment the current prefix before exploring children and decrement after backtracking to keep the scope local to the active path.


## Detailed Explanation
We perform a depth-first traversal starting from the root. At each node we compute the running prefix sum along the path from the root to the current node. Any ancestor path that ends at this node contributes to the target if `prefixCurrent - targetSum` exists in the hash map of previously seen prefix sums.

Algorithm steps:
1. Initialize a hash map `prefix_count` with `{0: 1}` so paths that start at the current node are counted when prefix equals target.
2. When visiting a node, add its value to the running sum.
3. Increase the answer by `prefix_count[running_sum - targetSum]` because those prefixes complete a target path ending here.
4. Increment `prefix_count[running_sum]` before exploring children, so descendants know this prefix exists.
5. Recurse into left and right children.
6. After returning, decrement `prefix_count[running_sum]` to remove the current node from the active path, preventing cross-branch contamination.

This approach ensures each node is processed once and the hash map mirrors the call stack state. It avoids enumerating all node pairs, reducing complexity from O(n^2) to O(n).


## Complexity Trade-off Table
| Approach | Time Complexity | Space Complexity |
| --- | --- | --- |
| Naive DFS exploring every start node | O(n^2) worst case | O(h) |
| DFS with prefix sum hash map | O(n) | O(h) recursion + O(h) hash map |


## Reference Implementation


In [ ]:
from collections import defaultdict
from typing import Optional


class TreeNode:
    def __init__(self, val: int = 0, left: Optional['TreeNode'] = None, right: Optional['TreeNode'] = None):
        self.val = val
        self.left = left
        self.right = right


class Solution:
    def pathSum(self, root: Optional[TreeNode], targetSum: int) -> int:
        prefix_count = defaultdict(int)
        prefix_count[0] = 1  # Empty path provides baseline for paths starting at current node.

        def dfs(node: Optional[TreeNode], running_sum: int) -> int:
            if not node:
                return 0

            running_sum += node.val
            total_paths = prefix_count[running_sum - targetSum]
            prefix_count[running_sum] += 1  # Expose current prefix to descendants.

            total_paths += dfs(node.left, running_sum)
            total_paths += dfs(node.right, running_sum)

            prefix_count[running_sum] -= 1  # Backtrack to keep state local to current path.
            return total_paths

        return dfs(root, 0)


## Complexity Analysis
- Time Complexity: O(n) because each node is visited once and each visit performs O(1) hash map operations.
- Space Complexity: O(h) for the recursion stack plus O(h) for the hash map in the worst case of a skewed tree of height h.
- Bottlenecks: Maintaining the prefix sum map during traversal is the dominant cost but remains linear.


## Edge Cases & Pitfalls
- Empty trees should return 0.
- Negative node values cannot be ignored; pruning based on sign will miss valid paths.
- Always decrement the prefix count when unwinding recursion to avoid counting paths across sibling branches.


## Follow-up Variants
- Extend to count paths whose sum falls within an inclusive range [L, R].
- Modify to return the actual node sequences for auditing purposes.
- Adapt the technique to directed acyclic graphs with a single parent for each node.


## Takeaways
- Prefix sums extend naturally from arrays to trees when paired with recursion.
- Shared mutable state must be carefully managed to avoid double counting.
- Pre-seeding the map with 0 elegantly covers paths that start at the current node.


## Similar Problems
| Problem ID | Problem Title | Technique |
| --- | --- | --- |
| 112 | Path Sum | DFS with running sums |
| 560 | Subarray Sum Equals K | Prefix sums with hash map |
| 124 | Binary Tree Maximum Path Sum | Recursive accumulation |
